# BERT
This notebook aims to use transfer learning on a BERT model to perform text classification and detect suicidal text.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -qqq transformers datasets wandb

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 290 kB 50.9 MB/s 
     |████████████████████████████████| 1.7 MB 39.8 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 31.7 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
     |████████████████████████████████| 3.3 MB 29.2 MB/s 
     |████████████████████████████████| 132 kB 48.6 MB/s 
     |████████████████████████████████| 243 kB 46.7 MB/s 
     |████████████████████████████████| 1.1 MB 29.8 MB/s 
     |████████████████████████████████| 139 kB 47.7 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 180 kB 44.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 160 kB 46.5 MB/s 
     |████████████████████████████████| 271 kB 46.2 MB/s 
     |████████████████████████████████| 192 kB 46.4 MB/s 


In [3]:
# Import packages
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

In [4]:
# Specify GPU
device = torch.device("cuda")

In [5]:
# Change to your own directory
try:
    os.chdir("/content/drive/MyDrive/BT4222 Project")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

Directory changed


## Define constants

In [6]:
# Define constants
EPOCHS = 1
BATCH_SIZE = 6
LEARNING_RATE = 1e-5
SEED = 4222

MODEL_SAVE_PATH = "Models/bert"
MODEL_CHECKPOINT_PATH = "Models/bert_checkpoint"
MODEL_LOGGING_PATH = "Models/bert_checkpoint/logs"

WANDB_ENTITY = "dennisrkibet"
WANDB_PROJECT = "suicide_detection"
WANDB_RUN = "bert"

## Load dataset

In [7]:
# Load dataset
df = pd.read_csv('Data/suicide_detection_final_cleaned.csv', header=0, names=['label', 'text', 'cleaned_text'])
df.drop(columns=['cleaned_text'], inplace=True)
df['label'] = df['label'].map({'suicide': 1, 'non-suicide': 0})
df.head()

,label,text
0,1,Ex Wife Threatening SuicideRecently I left my ...
1,0,Am I weird I don't get affected by compliments...
2,0,Finally 2020 is almost over... So I can never ...
3,1,i need helpjust help me im crying so hard
4,1,It ends tonight.I can’t do it anymore. \nI quit.


In [8]:
# Split dataset into train, validation and test sets
train, temp = train_test_split(df,
                               random_state=SEED,
                               test_size=0.2,
                               stratify=df['label'])

val, test = train_test_split(temp,
                             random_state=SEED,
                             test_size=0.5,
                             stratify=temp['label'])

## Load BERT Model

In [9]:
# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [10]:
def dataset_conversion(train, test, val):
  """Converts pandas dataframe to Dataset."""

  train.reset_index(drop=True, inplace=True)
  test.reset_index(drop=True, inplace=True)
  val.reset_index(drop=True, inplace=True)

  train_dataset = Dataset.from_pandas(train)
  test_dataset = Dataset.from_pandas(test)
  val_dataset = Dataset.from_pandas(val)

  return DatasetDict({"train": train_dataset,
                      "test": test_dataset,
                      "val": val_dataset})

raw_datasets = dataset_conversion(train, test, val)

In [11]:
def tokenize_function(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/140 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

In [12]:
# Tokenise datasets
SAMPLE_SIZE = 20
small_train_dataset = tokenized_datasets["train"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
small_val_dataset = tokenized_datasets["val"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))

full_train_dataset = tokenized_datasets["train"]
full_test_dataset = tokenized_datasets["test"]
full_val_dataset = tokenized_datasets["val"]

In [13]:
# Import BERT-base pretrained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
# Login wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [15]:
# Initialise wandb
wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY, name=WANDB_RUN)

wandb: Currently logged in as: gohjiayi (use `wandb login --relogin` to force relogin)
wandb: WARNING Cannot find valid git repo associated with this directory.


In [16]:
# Define custom metrics for computation
def compute_metrics(eval_pred):
    metric_acc = load_metric("accuracy")
    metric_rec = load_metric("recall")
    metric_pre = load_metric("precision")
    metric_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    recall = metric_rec.compute(predictions=predictions, references=labels)["recall"]
    precision = metric_pre.compute(predictions=predictions, references=labels)["precision"]
    f1 = metric_f1.compute(predictions=predictions, references=labels)["f1"]

    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [17]:
# Define model and training parameters
training_args = TrainingArguments(
    output_dir=MODEL_CHECKPOINT_PATH,
    overwrite_output_dir = True,
    report_to = 'wandb',
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    seed=SEED,
    # evaluation_strategy="epoch",
    run_name=WANDB_RUN,
    logging_dir=MODEL_LOGGING_PATH,
    save_strategy="steps",
    save_steps=1500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## Pre-trained BERT

In [ ]:
# Predict before fine-tuning
trainer.predict(full_test_dataset).metrics

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 17444
  Batch size = 6


Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

{'test_accuracy': 0.4680692501719789,
 'test_f1': 0.5743773221411862,
 'test_loss': 0.6981069445610046,
 'test_precision': 0.4155990706936608,
 'test_recall': 0.9294833729216152,
 'test_runtime': 1210.8379,
 'test_samples_per_second': 14.407,
 'test_steps_per_second': 2.402}

## Fine-tuned BERT

In [18]:
# %%wandb # To observe training progress live

# Fine-tune model
# trainer.train()

# Resume fine-tuning from checkpoint
trainer.train(MODEL_CHECKPOINT_PATH + "/" + "checkpoint-10500")

Loading model from Models/bert_checkpoint/checkpoint-10500).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 139548
  Num Epochs = 1
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 23258
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 10500
  Will skip the first 0 epochs then the first 10500 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/10500 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
11000,0.118300
11500,0.108500
12000,0.097100
12500,0.104700
13000,0.113600
13500,0.100200
14000,0.122200
14500,0.119600
15000,0.110300
15500,0.121100


Saving model checkpoint to Models/bert_checkpoint/checkpoint-12000
Configuration saved in Models/bert_checkpoint/checkpoint-12000/config.json
Model weights saved in Models/bert_checkpoint/checkpoint-12000/pytorch_model.bin
tokenizer config file saved in Models/bert_checkpoint/checkpoint-12000/tokenizer_config.json
Special tokens file saved in Models/bert_checkpoint/checkpoint-12000/special_tokens_map.json
Saving model checkpoint to Models/bert_checkpoint/checkpoint-13500
Configuration saved in Models/bert_checkpoint/checkpoint-13500/config.json
Model weights saved in Models/bert_checkpoint/checkpoint-13500/pytorch_model.bin
tokenizer config file saved in Models/bert_checkpoint/checkpoint-13500/tokenizer_config.json
Special tokens file saved in Models/bert_checkpoint/checkpoint-13500/special_tokens_map.json
Saving model checkpoint to Models/bert_checkpoint/checkpoint-15000
Configuration saved in Models/bert_checkpoint/checkpoint-15000/config.json
Model weights saved in Models/bert_check

Step,Training Loss
11000,0.118300
11500,0.108500
12000,0.097100
12500,0.104700
13000,0.113600
13500,0.100200
14000,0.122200
14500,0.119600
15000,0.110300
15500,0.121100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=23258, training_loss=0.05752664363151548, metrics={'train_runtime': 14442.815, 'train_samples_per_second': 9.662, 'train_steps_per_second': 1.61, 'total_flos': 3.671662155337728e+16, 'train_loss': 0.05752664363151548, 'epoch': 1.0})

In [25]:
# Terminate wandb run
wandb.finish()

wandb: WARNING os.getcwd() no longer exists, using system temp directory


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇████
train/learning_rate,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁


In [19]:
# Save fine-tuned model
trainer.save_model(MODEL_SAVE_PATH)

Saving model checkpoint to Models/bert
Configuration saved in Models/bert/config.json
Model weights saved in Models/bert/pytorch_model.bin
tokenizer config file saved in Models/bert/tokenizer_config.json
Special tokens file saved in Models/bert/special_tokens_map.json


In [20]:
# Evaluate fine-tuned model
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 17444
  Batch size = 6


Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

{'epoch': 1.0,
 'eval_accuracy': 0.9756936482458152,
 'eval_f1': 0.9684758364312268,
 'eval_loss': 0.10511656850576401,
 'eval_precision': 0.9700625558534406,
 'eval_recall': 0.9668942992874109,
 'eval_runtime': 1141.7676,
 'eval_samples_per_second': 15.278,
 'eval_steps_per_second': 2.547}

In [21]:
# Predict after fine-tuning
trainer.predict(full_test_dataset).metrics

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 17444
  Batch size = 6


{'test_accuracy': 0.9782160055033249,
 'test_f1': 0.9717640065388616,
 'test_loss': 0.09333978593349457,
 'test_precision': 0.9727759595358524,
 'test_recall': 0.9707541567695962,
 'test_runtime': 1140.4054,
 'test_samples_per_second': 15.296,
 'test_steps_per_second': 2.55}

In [24]:
def get_training_history(wandb_run):
  """Extract key metrics from training and eval from wandb run data."""

  # Get training history from wandb
  api = wandb.Api()
  run = api.run(wandb_run)
  history = run.history()

  # Rename columns
  train_column_dict = {'train/epoch': 'epoch', 'train/loss': 'training_loss'}
  val_column_dict = {'train/epoch': 'epoch', 'eval/loss': 'validation_loss', 'eval/accuracy': 'accuracy',
                'eval/precision': 'precision', 'eval/recall': 'recall', 'eval/f1': 'f1'}

  # Train data
  train_history = history[list(train_column_dict.keys())]
  train_history.columns = [train_column_dict.get(x, x) for x in train_history.columns]
  train_history = train_history.dropna()

  # Val data
  val_history = history[list(val_column_dict.keys())]
  val_history.columns = [val_column_dict.get(x, x) for x in val_history.columns]
  val_history = val_history.dropna()

  return pd.merge(train_history, val_history, how="right", on="epoch")

# Get dataframe for training history
WANDB_RUN_ID = "2bs164bd" # Replace with your wandb run details, found in the training cell

training_history = get_training_history(WANDB_ENTITY + "/" + WANDB_PROJECT + "/" + WANDB_RUN_ID)
training_history

,epoch,training_loss,validation_loss,accuracy,precision,recall,f1
0,1.0,NaN,0.105117,0.975694,0.970063,0.966894,0.968476


In [23]:
# Load fine-tuned model
saved_model = AutoModelForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

# Load trainer after fine-tune
saved_trainer = Trainer(
    model=saved_model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Predict after fine-tuning
saved_trainer.predict(full_test_dataset).metrics

loading configuration file Models/bert/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file Models/bert/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were ini

{'test_accuracy': 0.9782160055033249,
 'test_f1': 0.9717640065388616,
 'test_loss': 0.09333978593349457,
 'test_precision': 0.9727759595358524,
 'test_recall': 0.9707541567695962,
 'test_runtime': 1168.4957,
 'test_samples_per_second': 14.929,
 'test_steps_per_second': 2.489}

## GPU Memory Utilities

In [ ]:
# Delete variables and empty cache
del trainer
del model
torch.cuda.empty_cache()

In [ ]:
# Python garbage collection
import gc
gc.collect()

5236

In [ ]:
# Check memory allocation
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

0
11368660992


In [ ]:
# Check memory summary
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |   10776 MB |   16682 MB |   16682 MB |
|       from large pool |       0 B  |   10774 MB |   16680 MB |   16680 MB |
|       from small pool |       0 B  |       2 MB |       2 MB |       2 MB |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |   10776 MB |   16682 MB |   16682 MB |
|       from large pool |       0 B  |   10774 MB |   16680 MB |

In [ ]:
# Check GPU allocation and acprocesses
!nvidia-smi

Thu Nov 11 09:44:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    60W / 149W |  11429MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------